# Financial Machine Learning - Imbalance Bars 

上次介绍了对数据源进行加工处理的一些方式，包括了tick bars, volume bars以及dollar bars。使用这些bar能改进对数据的采样，并能保留更多的信息。对源数据的深度加工处理是一项非常关键的工作。一般来说，如果某项数据很容易获得，那么使用这些数据带来的价值不会很大，因为基本上大家都在用。比如普通的K线。我个人还是比较认可上述这个观点的。因此，对源数据的深入加工，发掘出更大的价值就显得极为重要了。

这篇将讨论Imbalance Bar，由于相对复杂，因此单独开一篇出来。观察市场，有一种所谓的“聪明钱”，这类资金通常掌握了关键信息（信息不对称），“聪明钱”要么大量买入，要么大量卖出，不太可能同时进行这两个操作。Imbalance bars 的目标就是当市场出现买入/卖出不均衡时对数据进行采样，也就是说，每次采样都是因为出现了不均衡的买卖操作。

## Dollar Imbalance Bars

总体的步骤可以分为：

1. 计算资金流向（流入/流出）

- 判断tick的方向；将当前tick price ( $P_1$ ) 同上一个tick price ( $P_0$ )比较，如果 $ P_1 > P_0 $ 则可视作资金流入，反之则为流出

- 将tick的方向和tick的量（volume）或金额相乘得到带方向的或金额

2. 动态生成Imbalance bar

- 从第一个tick开始，不断将带方向的成交量/金额汇总，这个值称为Imbalance

- 当Imbalance的绝对值超过了一定的阈值，则采样即生成一根bar

- 根据当前以获得的tick数据更新Imbalance的阈值，注意Imbalance的阈值是动态的

### 判断tick方向

例如我们获得了一个tick的序列，该序列包含了N个ticks${ (p[i], v[i]) }$ for $i ∈ 1…N$，其中 $p[i]$ 是价格，$v[i]$是成交金额。我们首先依次计算tick之间的价差，根据价差正负判断tick方向。并将该方向存入${b[i]}$ for $i ∈ 1…N$

$$ 

Δp[i] := p[i]-p[i-1] \\

b[i] := b[i-1]  \text{ if $Δp[i] = 0$} \\

b[i] := sign(Δp[i])    \text{ otherwise } \\

$$

如果如果2个连续的tick价格没有发生改变，则$Δp[i]=0$，那么交易方向$b[i]$保持不变即同$b[i-1]$。

